**1:1 Line (Diagonal/Identity Line):**

* This is a straight line with a slope of exactly 1 (45 degrees) where y = x
* It represents perfect predictions where the predicted value equals the actual value
* Any point on this line means the model predicted exactly the actual value
* Points above this line indicate the model is overpredicting
* Points below this line indicate the model is underpredicting
* In an ideal case, all predictions would fall exactly on this line


**Regression Line (Red Line):**

* This is the line of best fit through the scatter points
* It shows the actual relationship between predictions and actual values
* Its equation is y = mx + b, where:

* m is the slope (showing if predictions scale proportionally with actuals)
* b is the y-intercept (showing systematic bias in predictions)


**Comparing it to the 1:1 line tells us about model bias:**

* If it's parallel to 1:1 line but shifted: systematic bias
* If it has different slope: model tends to over/underpredict differently at different values


**In our last plots:**

* The RF model's regression line is closest to the 1:1 line, suggesting better predictions
* The LGB model shows more deviation, especially at higher values
* When regression line crosses the 1:1 line, it means the model overpredicts in one range and underpredicts in another


**This comparison helps assess:**

* Model Bias (systematic errors)
* Prediction Accuracy
* Range-specific performance
* Whether errors are consistent across the range or vary with the magnitude of the values


In [1]:
import os

In [2]:
from google.colab import drive
CONTENT_DRIVE = '/content/gdrive'
drive.mount(CONTENT_DRIVE, force_remount=True)

Mounted at /content/gdrive


In [3]:
HOME = os.path.join(CONTENT_DRIVE,'My Drive', 'Upwork_tasks', '2023_09_06_Remote_Sensing_Researcher', '2024_11_18_UPDATED_TASK')
os.listdir(HOME)

['Correlation - Mate Data.xlsx',
 'cao2021.pdf',
 'district_polygons.geojson',
 'csv_results',
 'plot_results',
 'results.csv',
 'correlation_plot_results',
 'plot_map_green_color.png',
 'comparison_RVIS_ML_Models.ipynb',
 '3_boost_Model.ipynb']

In [4]:
# Step 1: Install required libraries
!pip install xgboost lightgbm catboost scikit-learn pandas openpyxl folium pandas branca numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [5]:
!pip install plotly==5.24.1

In [6]:
# Step 2: Import required libraries
import copy
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import shapely
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [7]:
# Step 3: Load the dataset
file_path = os.path.join(HOME, 'Correlation - Mate Data.xlsx')   # Replace with your file path in Colab
data = pd.read_excel(file_path, sheet_name='Sheet1')
data.head()

Dist_Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [8]:
# strip potential empty spaces in the column names
data.columns = data.columns.str.strip()

In [9]:
# the Latitude is actually the Longitude and the Longitude is the Latitude (we gave to rename the columns)

data.rename(columns={'Latitude': 'Longitude', 'Longitude': 'Latitude'}, inplace=True)
data

Dist_Name  Longitude   Latitude  year  Area.in.Acres  Yield_kg/sq.km  \
0          DHAKA  90.255710  23.797116  2006       115091.0     456135.5637   
1       FARIDPUR  89.837466  23.476046  2006        58907.0     399370.5830   
2        GAZIPUR  90.449893  24.111209  2006       140143.0     426285.2461   
3      GOPALGANJ  89.900043  23.096562  2006       188628.0     433831.0846   
4    KISHOREGANJ  90.944409  24.372420  2006       283374.0     443078.6821   
..           ...        ...        ...   ...            ...             ...   
229  NARAYANGANJ  90.575194  23.740226  2023        48062.0     423649.9800   
230    NARSINGDI  90.775978  24.004128  2023       134037.0     377990.5000   
231      RAJBARI  89.562040  23.724970  2023        31865.0     451466.1300   
232   SHARIATPUR  90.412093  23.256006  2023        65086.0     417875.9900   
233      TANGAIL  89.997683  24.355739  2023       420016.0     433480.0700   

        RVI_1     RVI_2     RVI_3     RVI_4  ...      VV_6      VV_7  \
0    0.115295  0.168916  0.162720  0.147029  ...  0.088999  0.086428   
1    0.133071  0.120473  0.133402  0.131196  ...  0.074105  0.063211   
2    0.074087  0.131233  0.129388  0.134513  ...  0.064806  0.094200   
3    0.057413  0.116965  0.156272  0.146913  ...  0.102129  0.062042   
4    0.133673  0.139616  0.116582  0.133251  ...  0.101490  0.037176   
..        ...       ...       ...       ...  ...       ...       ...   
229  0.048182  0.151523  0.155045  0.148223  ...  0.072721  0.089457   
230  0.040464  0.125624  0.131704  0.122789  ...  0.070037  0.073739   
231  0.042975  0.139553  0.138332  0.150937  ...  0.073673  0.082826   
232  0.051608  0.167597  0.169685  0.166827  ...  0.066802  0.067873   
233  0.042312  0.167509  0.165459  0.154655  ...  0.074295  0.067075   

         VV_8      VV_9     VV_10     VV_11     VV_12  tmax_mean  tmin_mean  \
0    0.066533  0.115310  0.170770  0.094841  0.135790  44.273640  30.717323   
1    0.021454  0.124300  0.111684  0.108880  0.112395  41.469828  26.884820   
2    0.107330  0.082631  0.187207  0.128736  0.080302  43.172052  30.538223   
3    0.089569  0.131946  0.110374  0.110811  0.102837  44.345367  30.146326   
4    0.110393  0.085754  0.098181  0.115154  0.170666  42.809446  30.185885   
..        ...       ...       ...       ...       ...        ...        ...   
229  0.084466  0.108717  0.090104  0.108717  0.090104  44.330480  31.300754   
230  0.072117  0.092510  0.086482  0.092510  0.086482  42.851682  30.034350   
231  0.076823  0.138094  0.121817  0.138094  0.121817  45.848301  31.171559   
232  0.066360  0.104338  0.094413  0.104338  0.094413  44.299817  31.129609   
233  0.069446  0.101905  0.111385  0.101905  0.111385  44.559031  30.642676   

     humidity_mean  
0       110.801279  
1       103.953674  
2       104.135613  
3       106.689508  
4       105.824501  
..             ...  
229     104.592336  
230     101.773729  
231     104.593347  
232     106.908936  
233     106.852771  

[234 rows x 57 columns]

In [10]:
# Sort the data first by 'Dist_Name' and then by 'year'
data = data.sort_values(by=['year', 'Dist_Name'])
data

Dist_Name  Longitude   Latitude  year  Area.in.Acres  Yield_kg/sq.km  \
0          DHAKA  90.255710  23.797116  2006       115091.0     456135.5637   
1       FARIDPUR  89.837466  23.476046  2006        58907.0     399370.5830   
2        GAZIPUR  90.449893  24.111209  2006       140143.0     426285.2461   
3      GOPALGANJ  89.900043  23.096562  2006       188628.0     433831.0846   
4    KISHOREGANJ  90.944409  24.372420  2006       283374.0     443078.6821   
..           ...        ...        ...   ...            ...             ...   
229  NARAYANGANJ  90.575194  23.740226  2023        48062.0     423649.9800   
230    NARSINGDI  90.775978  24.004128  2023       134037.0     377990.5000   
231      RAJBARI  89.562040  23.724970  2023        31865.0     451466.1300   
232   SHARIATPUR  90.412093  23.256006  2023        65086.0     417875.9900   
233      TANGAIL  89.997683  24.355739  2023       420016.0     433480.0700   

        RVI_1     RVI_2     RVI_3     RVI_4  ...      VV_6      VV_7  \
0    0.115295  0.168916  0.162720  0.147029  ...  0.088999  0.086428   
1    0.133071  0.120473  0.133402  0.131196  ...  0.074105  0.063211   
2    0.074087  0.131233  0.129388  0.134513  ...  0.064806  0.094200   
3    0.057413  0.116965  0.156272  0.146913  ...  0.102129  0.062042   
4    0.133673  0.139616  0.116582  0.133251  ...  0.101490  0.037176   
..        ...       ...       ...       ...  ...       ...       ...   
229  0.048182  0.151523  0.155045  0.148223  ...  0.072721  0.089457   
230  0.040464  0.125624  0.131704  0.122789  ...  0.070037  0.073739   
231  0.042975  0.139553  0.138332  0.150937  ...  0.073673  0.082826   
232  0.051608  0.167597  0.169685  0.166827  ...  0.066802  0.067873   
233  0.042312  0.167509  0.165459  0.154655  ...  0.074295  0.067075   

         VV_8      VV_9     VV_10     VV_11     VV_12  tmax_mean  tmin_mean  \
0    0.066533  0.115310  0.170770  0.094841  0.135790  44.273640  30.717323   
1    0.021454  0.124300  0.111684  0.108880  0.112395  41.469828  26.884820   
2    0.107330  0.082631  0.187207  0.128736  0.080302  43.172052  30.538223   
3    0.089569  0.131946  0.110374  0.110811  0.102837  44.345367  30.146326   
4    0.110393  0.085754  0.098181  0.115154  0.170666  42.809446  30.185885   
..        ...       ...       ...       ...       ...        ...        ...   
229  0.084466  0.108717  0.090104  0.108717  0.090104  44.330480  31.300754   
230  0.072117  0.092510  0.086482  0.092510  0.086482  42.851682  30.034350   
231  0.076823  0.138094  0.121817  0.138094  0.121817  45.848301  31.171559   
232  0.066360  0.104338  0.094413  0.104338  0.094413  44.299817  31.129609   
233  0.069446  0.101905  0.111385  0.101905  0.111385  44.559031  30.642676   

     humidity_mean  
0       110.801279  
1       103.953674  
2       104.135613  
3       106.689508  
4       105.824501  
..             ...  
229     104.592336  
230     101.773729  
231     104.593347  
232     106.908936  
233     106.852771  

[234 rows x 57 columns]

In [11]:
# Step 4: Preprocess the data
# Define target and features
X = data.drop(columns=['Yield_kg/sq.km'])
y = data['Yield_kg/sq.km']

print(X.head())
print("--------------")
print(y.head())

     Dist_Name  Longitude   Latitude  year  Area.in.Acres     RVI_1     RVI_2  \
0        DHAKA  90.255710  23.797116  2006       115091.0  0.115295  0.168916   
1     FARIDPUR  89.837466  23.476046  2006        58907.0  0.133071  0.120473   
2      GAZIPUR  90.449893  24.111209  2006       140143.0  0.074087  0.131233   
3    GOPALGANJ  89.900043  23.096562  2006       188628.0  0.057413  0.116965   
4  KISHOREGANJ  90.944409  24.372420  2006       283374.0  0.133673  0.139616   

      RVI_3     RVI_4     RVI_5  ...      VV_6      VV_7      VV_8      VV_9  \
0  0.162720  0.147029  0.143458  ...  0.088999  0.086428  0.066533  0.115310   
1  0.133402  0.131196  0.110053  ...  0.074105  0.063211  0.021454  0.124300   
2  0.129388  0.134513  0.149914  ...  0.064806  0.094200  0.107330  0.082631   
3  0.156272  0.146913  0.161017  ...  0.102129  0.062042  0.089569  0.131946   
4  0.116582  0.133251  0.107605  ...  0.101490  0.037176  0.110393  0.085754   

      VV_10     VV_11     VV_12 

In [12]:
# Identify categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Define preprocessing for categorical and numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),  # OneHotEncoder for categorical features
        ('num', StandardScaler(), X.select_dtypes(exclude=['object']).columns)  # StandardScaler for numerical features
    ]
)

# Apply the preprocessor to features
X_preprocessed = preprocessor.fit_transform(X)
print(X_preprocessed.shape)


(234, 81)


In [60]:
# Step 6: Define a function to evaluate models with multiple metrics
def evaluate_model(model, model_name, X_train, X_test, y_train, y_test, year):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # make a copy of the y_pred
    y_pred_copy = copy.copy(y_pred)

    # y_test and y_pred in tonnes
    y_test = y_test / 1000
    y_pred = y_pred / 1000

    # Computation of row-wise RMSE (in tonnes)
    y_rmse_true_pred = np.sqrt((y_test.to_numpy() - y_pred)**2)

    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    # mse = mean_squared_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # create a dictionary for the results
    results = {
        'Model': model_name,
        'MAE': mae,
        # 'MSE': mse,
        'RMSE': rmse,
        'R2': r2,
        'MAPE': mape,
        'Year': year
    }

    return results, y_pred_copy, y_rmse_true_pred

In [61]:
# the following 5 sliding windows will be used:
#
# - years 2006 to year 2018 will be used to train the data and the year 2019 as test
# - years 2006 to year 2019 will be used to train the data and the year 2020 as test
# - years 2006 to year 2020 will be used to train the data and the year 2021 as test
# - years 2006 to year 2021 will be used to train the data and the year 2022 as test
# - years 2006 to year 2022 will be used to train the data and the year 2023 as test

seed_num = 1
sliding_window_test_res = []
RF, XG, LGB, CAT = {}, {}, {}, {}
RF_rmse, XG_rmse, LGB_rmse, CAT_rmse = {}, {}, {}, {}

reference_test_coords = X[X['year'] == 2018][['Dist_Name', 'Latitude', 'Longitude']]

# remove index from reference_test_coords
reference_test_coords.reset_index(drop=True, inplace=True)

# strip the names of the 'Dist_Name' column
reference_test_coords['Dist_Name'] = reference_test_coords['Dist_Name'].str.strip()


for train_end_year in range(2018, 2023):

  print(f"Train up to year {train_end_year}")

  # create the train and test sets for X
  x_year_train = X_preprocessed[X['year'] <= train_end_year]

  x_test_year = train_end_year + 1
  x_year_test = X_preprocessed[X['year'] == x_test_year]

  # get from X the 'Dist_Name', 'Latitude', 'Longitude'
  x_test_coords = X[X['year'] == x_test_year][['Dist_Name', 'Latitude', 'Longitude']]
  x_test_coords.reset_index(drop=True, inplace=True)
  x_test_coords['Dist_Name'] = x_test_coords['Dist_Name'].str.strip()
  if not reference_test_coords.equals(x_test_coords):
    raise ValueError(f"The 'Dist_Name', 'Latitude', 'Longitude' order of the observations of year '{x_test_year}' are different from the reference test coordinates!")

  # create the train and test sets for y
  y_year_train = y[X['year'] <= train_end_year]
  y_year_test = y[X['year'] == train_end_year + 1]

  # Random Forest
  rf_model = RandomForestRegressor(random_state=seed_num)
  rf_res, rf_preds, rf_rmse = evaluate_model(rf_model, "Random_Forest", x_year_train, x_year_test, y_year_train, y_year_test, x_test_year)
  sliding_window_test_res.append(rf_res)
  RF[x_test_year] = rf_preds
  RF_rmse[x_test_year] = rf_rmse

  # XGBoost
  xgb_model = XGBRegressor(random_state=seed_num)
  xg_res, xg_preds, xg_rmse = evaluate_model(xgb_model, "XGBoost", x_year_train, x_year_test, y_year_train, y_year_test, x_test_year)
  sliding_window_test_res.append(xg_res)
  XG[x_test_year] = xg_preds
  XG_rmse[x_test_year] = xg_rmse

  # LightGBM (omit progress)
  lgb_model = LGBMRegressor(random_state=seed_num, verbosity=-1)
  lgb_res, lgb_preds, lgb_rmse = evaluate_model(lgb_model, "LightGBM", x_year_train, x_year_test, y_year_train, y_year_test, x_test_year)
  sliding_window_test_res.append(lgb_res)
  LGB[x_test_year] = lgb_preds
  LGB_rmse[x_test_year] = lgb_rmse

  # CatBoost
  cat_model = CatBoostRegressor(verbose=0, random_state=seed_num)
  cb_res, cb_preds, cb_rmse = evaluate_model(cat_model, "CatBoost", x_year_train, x_year_test, y_year_train, y_year_test, x_test_year)
  sliding_window_test_res.append(cb_res)
  CAT[x_test_year] = cb_preds
  CAT_rmse[x_test_year] = cb_rmse

  seed_num += 1


Train up to year 2018
Train up to year 2019
Train up to year 2020
Train up to year 2021
Train up to year 2022


In [62]:
# create directory to save the results and each ml-model
dir_csv = os.path.join(HOME, 'csv_results')
if not os.path.exists(dir_csv):
  os.makedirs(dir_csv, exist_ok=True)

dir_plots = os.path.join(HOME, 'plot_results')
if not os.path.exists(dir_plots):
  os.makedirs(dir_plots, exist_ok=True)

dir_cor = os.path.join(HOME, 'correlation_plot_results')
if not os.path.exists(dir_cor):
  os.makedirs(dir_cor, exist_ok=True)

In [63]:
# save the evaluation metrics

df = pd.DataFrame(sliding_window_test_res)

# sort the dataframe by R2
df = df.sort_values(by='R2', ascending=False)

# write the dataframe to a .csv file
df.to_csv(os.path.join(HOME, 'results.csv'), index=False)

In [64]:
# for the 3 models create a dataframe by including the coordinates
def merge_coords(df, coords, file_save, convert_to_tones = True):
  if convert_to_tones:
    df = df / 1000
  df = pd.concat([df, coords], axis=1)
  df.to_csv(file_save, index=False)
  return df

# create a dataframe for each model
RF_df = merge_coords(pd.DataFrame(RF), reference_test_coords, os.path.join(dir_csv, 'RF.csv'))
XG_df = merge_coords(pd.DataFrame(XG), reference_test_coords, os.path.join(dir_csv, 'XG.csv'))
LGB_df = merge_coords(pd.DataFrame(LGB), reference_test_coords, os.path.join(dir_csv, 'LGB.csv'))
CAT_df = merge_coords(pd.DataFrame(CAT), reference_test_coords, os.path.join(dir_csv, 'CAT.csv'))

# create a dataframe for each model
RF_df_rmse = merge_coords(pd.DataFrame(RF_rmse), reference_test_coords, os.path.join(dir_csv, 'RF_rmse.csv'))
XG_df_rmse = merge_coords(pd.DataFrame(XG_rmse), reference_test_coords, os.path.join(dir_csv, 'XG_rmse.csv'))
LGB_df_rmse = merge_coords(pd.DataFrame(LGB_rmse), reference_test_coords, os.path.join(dir_csv, 'LGB_rmse.csv'))
CAT_df_rmse = merge_coords(pd.DataFrame(CAT_rmse), reference_test_coords, os.path.join(dir_csv, 'CAT_rmse.csv'))


In [65]:
RF_df

2019        2020        2021        2022        2023    Dist_Name  \
0   392.518466  512.668102  459.854279  498.370667  471.213940        DHAKA   
1   381.827614  456.799546  453.468665  443.426920  444.316521     FARIDPUR   
2   420.937291  410.637444  432.783521  431.837017  436.629086      GAZIPUR   
3   481.714708  511.449082  490.890643  501.852954  519.307974    GOPALGANJ   
4   427.442158  409.626099  430.443418  407.320846  418.177245  KISHOREGANJ   
5   418.148776  435.204564  429.424916  458.442997  433.849448    MADARIPUR   
6   444.723852  473.100988  483.233768  468.865402  513.810074    MANIKGANJ   
7   411.593641  428.350544  445.835257  451.679670  405.514942   MUNSHIGANJ   
8   391.866198  420.589736  430.412889  396.911902  418.110900  NARAYANGANJ   
9   404.919096  393.783434  391.000569  375.069147  375.013732    NARSINGDI   
10  389.056201  428.380145  416.190470  444.125477  444.444973      RAJBARI   
11  433.346944  476.702719  460.100085  443.385825  410.847994   SHARIATPUR   
12  434.270277  400.033308  432.772544  426.968384  431.532708      TANGAIL   

     Latitude  Longitude  
0   23.797116  90.255710  
1   23.476046  89.837466  
2   24.111209  90.449893  
3   23.096562  89.900043  
4   24.372420  90.944409  
5   23.213496  90.166085  
6   23.840500  89.995450  
7   23.527374  90.422476  
8   23.740226  90.575194  
9   24.004128  90.775978  
10  23.724970  89.562040  
11  23.256006  90.412093  
12  24.355739  89.997683

In [67]:
RF_df_rmse

2019      2020      2021      2022      2023    Dist_Name   Latitude  \
0   0.008350  0.001113  0.000848  0.003021  0.000082        DHAKA  23.797116   
1   0.000196  0.004604  0.008138  0.000668  0.005330     FARIDPUR  23.476046   
2   0.007005  0.004663  0.001087  0.002064  0.005350      GAZIPUR  24.111209   
3   0.001704  0.000200  0.001939  0.003054  0.041095    GOPALGANJ  23.096562   
4   0.002977  0.012274  0.005419  0.002173  0.006973  KISHOREGANJ  24.372420   
5   0.003239  0.005029  0.003889  0.005376  0.003454    MADARIPUR  23.213496   
6   0.004923  0.000834  0.000300  0.001995  0.003048    MANIKGANJ  23.840500   
7   0.009751  0.000452  0.004157  0.006489  0.001681   MUNSHIGANJ  23.527374   
8   0.006618  0.001363  0.000096  0.001715  0.005539  NARAYANGANJ  23.740226   
9   0.000284  0.001972  0.005145  0.008437  0.002977    NARSINGDI  24.004128   
10  0.000300  0.001081  0.008829  0.001532  0.007021      RAJBARI  23.724970   
11  0.002856  0.000833  0.004683  0.000065  0.007028   SHARIATPUR  23.256006   
12  0.001090  0.002588  0.000507  0.005610  0.001947      TANGAIL  24.355739   

    Longitude  
0   90.255710  
1   89.837466  
2   90.449893  
3   89.900043  
4   90.944409  
5   90.166085  
6   89.995450  
7   90.422476  
8   90.575194  
9   90.775978  
10  89.562040  
11  90.412093  
12  89.997683

In [68]:
fig = px.scatter_map(RF_df, lat="Latitude", lon="Longitude", color=2019, size=2019,
                     map_style="carto-positron", zoom=8, width=1200, height=800)
fig.show()


In [69]:
pth_geojson = os.path.join(HOME,'district_polygons.geojson' )

In [70]:
with open(pth_geojson) as f:
  features = json.load(f)

In [71]:
df = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth_map(RF_df,
                        geojson=features,
                        color=2019,
                        locations="Dist_Name",
                        featureidkey="properties.Dist_Name",
                        center={"lat": 23.83872, "lon": 90.24065},
                        #  color_continuous_scale="Blues",
                        color_continuous_scale="Greens",
                        #  color_continuous_scale="aggrnyl",
                        map_style="carto-positron",
                        zoom=8)

fig.update_layout(title="Bangladesh Agricultural Map")
fig.update_layout(width=1600, height=1000)

fig.show()

In [72]:
# parameters for the plots
models = ['RF', 'XGB', 'LGB', 'CAT']
titles = ['Random Forest', 'Gradient Boost Machine', 'LightGBM', 'CatBoost']


# Create density colors for points
def calculate_density(x, y, bins=20):
    hist2d, x_edges, y_edges = np.histogram2d(x, y, bins=bins)
    x_centers = (x_edges[:-1] + x_edges[1:]) / 2
    y_centers = (y_edges[:-1] + y_edges[1:]) / 2
    x_idx = np.digitize(x, x_edges) - 1
    y_idx = np.digitize(y, y_edges) - 1
    x_idx[x_idx == bins] = bins - 1
    y_idx[y_idx == bins] = bins - 1
    densities = hist2d[x_idx, y_idx]
    return densities / np.max(densities)


# create & save separate plots based on the RF_df, XG_df, LGB_df, CAT_df
for YEAR in range(2019, 2024):

  # subset the dataframes
  RF_df_subs = RF_df[YEAR]
  RF_df_subs.rename('RF', inplace=True)     # rename the column

  XG_df_subs = XG_df[YEAR]
  XG_df_subs.rename('XGB', inplace=True)

  LGB_df_subs = LGB_df[YEAR]
  LGB_df_subs.rename('LGB', inplace=True)

  CAT_df_subs = CAT_df[YEAR]
  CAT_df_subs.rename('CAT', inplace=True)

  # bind all columns
  data_plt = pd.concat([RF_df_subs, XG_df_subs, LGB_df_subs, CAT_df_subs], axis=1)

  # actual data of the year
  actual_values = y[X['year'] == YEAR] / 1000
  # actual_values = actual_values.values
  actual_values.rename('Actual', inplace=True)
  actual_values.reset_index(drop=True, inplace=True)
  data_plt = pd.concat([data_plt, actual_values], axis=1)

  # Create figure with 2x2 subplots
  fig, axes = plt.subplots(2, 2, figsize=(15, 15))
  fig.suptitle(f'Rice Production Predictions ML-Models year {YEAR}', fontsize=16)

  # Create scatter plots for each model
  for idx, (model, title) in enumerate(zip(models, titles)):
      row = idx // 2
      col = idx % 2
      ax = axes[row, col]

      # Calculate metrics
      actual = np.array(data_plt['Actual'])
      predicted = np.array(data_plt[model])
      correlation = np.corrcoef(actual, predicted)[0, 1]
      r2 = correlation ** 2
      rmse = np.sqrt(np.mean((actual - predicted) ** 2))

      # Calculate point densities
      densities = calculate_density(actual, predicted)

      # Create scatter plot with custom colormap
      scatter = ax.scatter(actual, predicted, c=densities,
                          cmap='viridis', alpha=0.6)

      # Add colorbar
      plt.colorbar(scatter, ax=ax, label='Density')

      # Add diagonal line (1:1 line)
      min_val = min(min(actual), min(predicted))
      max_val = max(max(actual), max(predicted))
      ax.plot([min_val, max_val], [min_val, max_val], 'k--', label='1:1 Line')

      # Add regression line
      z = np.polyfit(actual, predicted, 1)
      p = np.poly1d(z)
      ax.plot(actual, p(actual), 'r-', label='Regression Line')

      # Add metrics text
      ax.text(0.05, 0.95, f'Corr = {correlation:.2f}\nR² = {r2:.2f}\nRMSE = {rmse:.2f}',
              transform=ax.transAxes, fontsize=10, verticalalignment='top',
              bbox=dict(facecolor='white', alpha=0.8))

      # Set labels and title
      ax.set_xlabel('True Rice Production (kiloton)')
      ax.set_ylabel('Predicted Rice Production (kiloton)')
      ax.set_title(f'Rice Production Predictions with {title}')

      # Place legend in lower right corner
      ax.legend(loc='lower right', bbox_to_anchor=(0.95, 0.05))

      # Set axis limits
      ax.set_xlim(350, 500)
      ax.set_ylim(350, 500)

  # Adjust layout
  plt.tight_layout(rect=[0, 0.03, 1, 0.95])

  # Save the plot with high resolution
  plt.savefig(os.path.join(dir_plots, f'ml_year_{YEAR}' ), dpi=300, bbox_inches='tight')

  # # Show plot
  # plt.show()

  plt.close()


In [73]:
# subset the data to keep only the "Yield_kg/sq.km" column and the 12 RVI columns (RVI_1 to RVI_12)

df = data[['Yield_kg/sq.km'] + [f'RVI_{i}' for i in range(1, 13)]]
df.head()

Yield_kg/sq.km     RVI_1     RVI_2     RVI_3     RVI_4     RVI_5     RVI_6  \
0     456135.5637  0.115295  0.168916  0.162720  0.147029  0.143458  0.189104   
1     399370.5830  0.133071  0.120473  0.133402  0.131196  0.110053  0.140661   
2     426285.2461  0.074087  0.131233  0.129388  0.134513  0.149914  0.098708   
3     433831.0846  0.057413  0.116965  0.156272  0.146913  0.161017  0.142765   
4     443078.6821  0.133673  0.139616  0.116582  0.133251  0.107605  0.118882   

      RVI_7     RVI_8     RVI_9    RVI_10    RVI_11    RVI_12  
0  0.169951  0.131053  0.181035  0.144915  0.125618  0.155944  
1  0.114671  0.141824  0.116194  0.135737  0.122178  0.124351  
2  0.151873  0.169926  0.141616  0.130535  0.123264  0.138241  
3  0.148320  0.136261  0.153077  0.126849  0.156924  0.150205  
4  0.119803  0.151816  0.141380  0.143256  0.160030  0.132033

In [74]:

# Create a loop to generate plots for each RVI column
for i in range(1, 13):  # RVI_1 to RVI_12
    # Create a new figure for each plot
    plt.figure(figsize=(10, 6))

    # Create scatter plot
    plt.scatter(df[f'RVI_{i}'], df['Yield_kg/sq.km'], alpha=0.7)

    # Calculate and plot trend line
    z = np.polyfit(df[f'RVI_{i}'], df['Yield_kg/sq.km'], 1)
    p = np.poly1d(z)
    plt.plot(df[f'RVI_{i}'], p(df[f'RVI_{i}']), "g--", label='Trend Line')

    # Set labels and title
    plt.xlabel(f'RVI_{i} Value')
    plt.ylabel('Yield (kg/sq.km)')
    plt.title(f'Correlation between Yield and RVI_{i}')

    # Add legend
    plt.legend()

    # Add grid
    plt.grid(True, linestyle='--', alpha=0.7)

    # Save the plot
    plt.savefig(os.path.join(dir_cor, f'Yield_vs_RVI_{i}_Correlation.png'))

    # Close the plot to free up memory
    plt.close()

print("All correlation plots have been generated and saved.")

All correlation plots have been generated and saved.
